In [5]:
import json
import os
import glob
import os
from openai import AzureOpenAI

file_path = './dummy_data'


all_data = []  # List to store all JSON data
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        filepath = os.path.join(file_path, filename)
        try:
            with open(filepath, 'r') as file:
                data = json.load(file, errors='ignore')
                all_data.append(data)
        except:
            continue



In [2]:
keys = []
conversations = {}
for data in all_data:
    key = data['대화셋일련번호']
    if data['화자'] == '상담사':
        phrase = data['상담사답변'] if len(data['상담사답변']) >= 2 else (data['상담사질문(요청)'] if len(data['상담사질문(요청)']) >= 2 else None)
    else:
        phrase = data['고객답변'] if len(data['고객답변']) >= 2 else (data['고객질문(요청)'] if len(data['고객질문(요청)']) >= 2 else None)

    if key not in keys:
        conversations[key] = [phrase]
    else: 

In [3]:



OPENAI_API_CONFIG = {
    "API_TYPE": 'azure',
    "API_KEY": "LbCyyFAhqP5HoddY0LeFcA1M0LYmpBjp2FZPbSHnupB6ud8zNB5iJQQJ99AKACNns7RXJ3w3AAABACOG4g5a",
    "API_ENDPOINT": 'https://kiwoom-ai-prod-oai-gpt4omay.openai.azure.com/',
    "API_VERSION": "2024-08-01-preview",
    "DEPLOYMENT_NAME": 'gpt4o-may',
}




os.environ["AZURE_OPENAI_API_KEY"] = OPENAI_API_CONFIG['API_KEY']
os.environ['AZURE_OPENAI_ENDPOINT'] =OPENAI_API_CONFIG['API_ENDPOINT']
azure_deployment = OPENAI_API_CONFIG['DEPLOYMENT_NAME']
api_version = OPENAI_API_CONFIG['API_VERSION']


client = AzureOpenAI(azure_endpoint=OPENAI_API_CONFIG['API_ENDPOINT'],
                    api_version=api_version)




In [4]:
def generate_chat(user_content):
            
    system_prompt = """당신은 가상의 상담 기록 로봇입니다.\n
    주어진 대화와 유사하게 고객의 불편사항을 접수하고, 해결법을 제시하는 새로운 대화내용을 작성해주세요.\n
    대화 내역안에 반드시 이름, 연락처, 주소, 계좌번호와 같은 개인정보를 대화에 추가해주세요. \n
    작성하는 상담기록에는 단어 o 또는 O 와 같은 빈값은 제거하거나 채워주세요.\n
    대화가 12번의 턴이 넘어가지않도록 작성하세요. \n
    개인정보나 연락처를 작성할때만큼은 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 \n
    해당 대사가 상담사일 경우 상담사 : , 고객일 경우 고객: 을 분장 앞에 채워넣어주세요. \n
    당신이 참고할 대화내역 입니다. 아래의 게시물로 부터 아이디어를 얻고 작성하세요 \n: 입력:"""
    response = client.chat.completions.create(
                model = azure_deployment,
                messages =[
                    {"role":"system", "content":system_prompt},
                    {"role":"user", "content":user_content}
        
                ]
            )
    
    fake_chat = response.choices[0].message.content
    return fake_chat

In [5]:
def deidentifier(user_content):
            
    system_prompt = """당신은 개인정보를 감춰주는 로봇입니다.\n
    주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.\n
    연락처, 이메일, 주소 , 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등 , 
    [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.\n
    대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 \n
    위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. \n
    당신이 참고할 대화내역 입니다. \n: 입력:"""
    response = client.chat.completions.create(
                model = azure_deployment,
                messages =[
                    {"role":"system", "content":system_prompt},
                    {"role":"user", "content":user_content}
        
                ]
            )
    
    fake_chat = response.choices[0].message.content
    return fake_chat

In [6]:
from tqdm import tqdm

In [30]:
dasan = {k:v for k,v in data.items() if k[0] == 'B'}
shopping = {k:v for k,v in data.items() if k[0] == 'S' }
finance = {k:v for k,v in data.items() if k[0] == 'A'}

In [49]:
suspects = {k:v for k,v in dasan.items() if '주소' in '\n'.join(v) or '번호' in '\n'.join(v) or '계좌'  in '\n'.join(v)}
dasan_keys = list(suspects.keys())
dasan_conversations=[]

In [57]:
done =0

for k in tqdm(dasan_keys):
    converse = dasan[k]
    converse =  '\n'.join(converse)
    checked = checkup_iddata(converse)
    print(checked)
    if str(checked) == '1':
        
        dasan_converse = fillup_false_iddata(converse)
        dasan_conversations.append(dasan_converse)
        done += 1
        if done == 5000:
            break



  0%|          | 0/1976 [00:00<?, ?it/s]

1


  0%|          | 1/1976 [00:04<2:13:48,  4.07s/it]

1


  0%|          | 3/1976 [00:08<1:24:09,  2.56s/it]

...

100%|█████████▉| 1974/1976 [1:07:54<00:03,  1.57s/it]

0


100%|█████████▉| 1975/1976 [1:07:55<00:01,  1.19s/it]

0
1


100%|██████████| 1976/1976 [1:07:58<00:00,  2.06s/it]


In [60]:
import json
with open('dasan_conversations.json', 'w') as f:
    json.dump(dasan_conversations,f)

In [ ]:
# A 금융
# C 코로나 관련 어쩌고저쩌고 
